In [1]:
# it seems the tcgplayer api is no longer available for new people, so I will have to scrape some data on my own

In [2]:
from selenium import webdriver
import time
import math
import re
import itertools
import pandas as pd
import selenium.webdriver.support.ui as ui

In [100]:
# this is how to obtain a title
def get_title(html_source, item_number):
    regex_title = re.compile('{}(.*){}'.format(re.escape('search-result__title">'), re.escape('</span><div class="inventory"')))
    html_title = re.findall('search-result__title">[^<]+</span><div class="inventory"',html_source)
    title = regex_title.findall(html_title[item_number])
    return(title)

In [101]:
def get_table_data(html, title):
    # the text we're interested in is tbody then the dates and price that follows
    regex_table = re.compile('{}(.*){}'.format(re.escape('tbody'), re.escape('tbody')))
    html_table_1 = regex_table.findall(html)
    dates = re.findall('\d{1,2}/\d{1,2} to \d{1,2}/\d{1,2}',html_table_1[0])
    prices = re.findall('\$\d+(?:\.\d+)?',html_table_1[0])
    regex_foil = re.compile('{}(.*){}'.format(re.escape('<div class="charts-title" data-v-f69e3427="">'), re.escape('</div>')))
    html_foil = re.findall('<div class="charts-title" data-v-f69e3427="">[^<]+</div>',html)
    foil = [regex_foil.findall(html_foil[i]) for i in range(len(html_foil))]
    foils = foil * len(dates)
    if len(dates) < len(prices):
        r = int(len(prices)/len(dates))
        dates = [item for item in dates for _ in range(r)] 
    titles = title * len(dates)
    return titles, foils, dates, prices

In [102]:
url = 'https://www.tcgplayer.com/search/pokemon/sv-scarlet-and-violet-151?productLineName=pokemon&setName=sv-scarlet-and-violet-151&view=grid&page=1'
driver = webdriver.Chrome(executable_path=r"C:\Users\steve\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe")
driver.get(url)
time.sleep(7)
html_source = driver.page_source



# how to find number of search results
regex = re.compile('{}(.*){}'.format(re.escape('search-result-count'), re.escape(' results')))
shortened_html = regex.findall(html_source)
num_results = re.findall('^.*>([0-9]+)$',shortened_html[0])
num_results = int(num_results[0])
num_pages = math.ceil(num_results/24)


htmls = []
titles = []
dates = []
prices = []
foils = []

# Sort A-Z so the products stay in order
sort_toggle = driver.find_element_by_class_name('tcg-input-select__trigger-toggle-button')
sort_toggle.click() 
time.sleep(3)
xpath = '/html/body/div[2]/div/div/section[2]/section/div/div[2]/div/div/div[2]/div/div/div/div/ul/li[3]/span'
a_to_z_sort = driver.find_element_by_xpath(xpath)
a_to_z_sort.click()
    
time.sleep(10)

for h in range(num_pages):
    # go to the next page
    url = url[:-1] + str(h+1)
    driver.get(url)    

    time.sleep(12)
    item_titles = driver.find_elements_by_class_name('search-result__title')
    for i in range(len(item_titles)):
        html_source = driver.page_source
        title = get_title(html_source, i)
        
        item_titles = driver.find_elements_by_class_name('search-result__title')
#         print(h+1, i, len(item_titles))
        item_titles[i].click()
        time.sleep(12)
        html = driver.page_source
        htmls.append(html)
        
        title, foil, date, price = get_table_data(html, title)
#         if title[0] in titles:
#             print(title[0])
#         print(price[-1])
        titles.append(title)
        foils.append(foil)
        dates.append(date)
        prices.append(price)
        
        driver.back()
        time.sleep(12)


In [103]:
len(titles), len(dates), len(prices), len(foils)

(230, 230, 230, 230)

In [104]:
unique_titles = []
num = 0
# check to see that all titles are unique
for title in titles:
    if title[0] in unique_titles:
        print(title[0])
        print(num)
    else:
        unique_titles.append(title[0])
    num += 1
    

In [105]:
titles = list(itertools.chain(*titles))
dates = list(itertools.chain(*dates))
prices = list(itertools.chain(*prices))
foils = list(itertools.chain(*foils))

In [106]:
len(titles), len(dates), len(prices), len(foils)

(11520, 11520, 11520, 11520)

In [107]:
market_prices = pd.DataFrame([titles, foils, dates, prices]).T
market_prices = market_prices.rename(columns={0: 'Product', 1:'Foil', 2:'Date Range', 3: 'Market Price'})
market_prices

,Product,Foil,Date Range,Market Price
0,151: Alakazam ex Collection,[ Market Price: ],7/17 to 7/19,$31.98
1,151: Alakazam ex Collection,[ Market Price: ],7/20 to 7/22,$31.77
2,151: Alakazam ex Collection,[ Market Price: ],7/23 to 7/25,$31.21
3,151: Alakazam ex Collection,[ Market Price: ],7/26 to 7/28,$31.12
4,151: Alakazam ex Collection,[ Market Price: ],7/29 to 7/31,$29.44
...,...,...,...,...
11515,Zubat,[Reverse Holofoil Market Price: ],10/6 to 10/8,$0.23
11516,Zubat,[ Market Price: ],10/9 to 10/11,$0.06
11517,Zubat,[Reverse Holofoil Market Price: ],10/9 to 10/11,$0.21
11518,Zubat,[ Market Price: ],10/12 to 10/14,$0.04


In [108]:
len(market_prices['Product'].unique())

230

In [109]:
market_prices.to_csv('Pokemon_151.csv')